<a href="https://colab.research.google.com/github/Dromedara/AI-treaning/blob/main/code/Segmebtation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#1
%%shell

pip install cython
# Install pycocotools, the version by default in Colab
# has a bug fixed in https://github.com/cocodataset/cocoapi/pull/354
pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-y37giax4
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-y37giax4
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=265172 sha256=3c50478652c14f1c4a0a271b189bfd4bc6baac1af61a71d1365e0dc0cd3999e2
  Stored in directory: /tmp/pip-ephem-wheel-cache-do6qg2rn/wheels/e2/6b/1d/344ac773c7495ea0b85eb228bc66daec7400a143a92d36b7b1
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.4
    Uninstalling pycocotools-2.0.4:
      Successfully uninstalled pycocotools-2.0.4


In [ ]:
#2
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.8.2

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

Cloning into 'vision'...
remote: Enumerating objects: 177859, done.
remote: Counting objects: 100% (10244/10244), done.
remote: Compressing objects: 100% (672/672), done.
remote: Total 177859 (delta 9647), reused 10097 (delta 9551), pack-reused 167615
Receiving objects: 100% (177859/177859), 350.90 MiB | 16.88 MiB/s, done.
Resolving deltas: 100% (159253/159253), done.
Note: checking out 'v0.8.2'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 2f40a483d [v0.8.X] .circleci: Add Python 3.9 to CI (#3063)


In [ ]:
#3
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [98]:
#4
import torch.nn as nn
import cv2
import copy
import json

In [ ]:
#5
from PIL import Image, ImageDraw

def mask_from_coco(img, target):
    img = np.array(img)
    
    polygon = target['segmentation'][0]
    img = Image.new('L', (img.shape[1], img.shape[0]), 0)
    try:
      ImageDraw.Draw(img).polygon(polygon, outline=1, fill=1)
    except:
      print(target['image_id'])
      raise
      
    mask = np.array(img)
    
    return mask

In [ ]:
#6
import os
import numpy as np
import torch
from PIL import Image
import json


class ArucoDataset(torch.utils.data.Dataset):
    def __init__(self, root, annFile, transforms):
        self.root = root
        
        with open('drive/MyDrive/VisionProject/Dataset/Labels.json') as f:
          labels = json.load(f)

          self.annotations = list(filter(lambda x: len(x['segmentation'][0]) > 2, labels['annotations']))
          self.id2file = {
              meta['id']: meta['file_name']
              for meta in labels['images']
          }

        self.transforms = transforms
        
    def __getitem__(self, idx):
        if not isinstance(idx, int):
            idx = idx.item()

        target = copy.deepcopy(self.annotations[idx])
        
        # load images and masks
        img = cv2.imread(
            os.path.join(
                self.root, 
                self.id2file[target['image_id']]
            ),
            cv2.IMREAD_GRAYSCALE
        )

        mask = mask_from_coco(img, target)
        
        target['bbox'][2] += target['bbox'][0]
        target['bbox'][3] += target['bbox'][1]
        
        boxes = torch.Tensor([target['bbox']])
        
        target['masks'] = torch.as_tensor([mask], dtype=torch.uint8)
        target['boxes'] = boxes
        target['labels'] = torch.tensor([1], dtype=torch.int64)
        target['area'] = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        target['iscrowd'] = torch.tensor([target['iscrowd']])
        target['image_id'] = torch.tensor([idx])
        target['id'] = torch.tensor([idx])

        del target['category_id']
        del target['segmentation']

        for key, value in target.items():
            target[key] = torch.tensor(value)
            
        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.annotations)

In [ ]:
#7
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [ ]:
#8
import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [ ]:
#9
import utils

In [ ]:
#10

# use our dataset and defined transformations
dataset = ArucoDataset('drive/MyDrive/VisionProject/Dataset/Data/', 'drive/MyDrive/VisionProject/Dataset/Labels.json', get_transform(train=True))
dataset_test = ArucoDataset('drive/MyDrive/VisionProject/Dataset/Data/', 'drive/MyDrive/VisionProject/Dataset/Labels.json', get_transform(train=False))

# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()

test_size = 320

dataset = torch.utils.data.Subset(dataset, indices[:-test_size])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-test_size:])

print(len(dataset), len(dataset_test))

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False,
    collate_fn=utils.collate_fn)

In [ ]:
# 11

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 2

# get the model using our helper function
model = get_instance_segmentation_model(num_classes)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

In [ ]:
# 12

from engine import train_one_epoch, evaluate

In [ ]:
# 13

# let's train it for 10 epochs
from torch.optim.lr_scheduler import StepLR
num_epochs = 10

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

In [ ]:
# 14

from torchvision.utils import draw_bounding_boxes
import matplotlib.pyplot as plt

In [ ]:
# 15

ls


coco_eval.py   drive/     __pycache__/  transforms.py  vision/
coco_utils.py  engine.py  sample_data/  utils.py


In [ ]:
# 16

torch.save(model.state_dict(), '/content/drive/MyDrive/VisionProject/Model/segmentation_model.pt')
model.eval();

In [ ]:
# 17

import glob

In [ ]:
# 18

dataset.dataset.id2file.values()

In [ ]:
# 19

glob.glob('drive/MyDrive/VisionProject/Dataset/Data/*')[0]

'drive/MyDrive/VisionProject/Dataset/Data/0088.jpg'

In [ ]:
# 20

other_images = set(glob.glob('drive/MyDrive/VisionProject/Dataset/Data/*'))
other_images.difference_update(list(map(lambda x: 'drive/MyDrive/VisionProject/Dataset/Data/' + x, dataset.dataset.id2file.values())))
other_images.difference_update(list(map(lambda x: 'drive/MyDrive/VisionProject/Dataset/Data/' + x, dataset_test.dataset.id2file.values())))
other_images = list(other_images)

In [ ]:
# 21

import matplotlib.pyplot as plt

In [ ]:
# 22

!pip install imantics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for imantics: filename=imantics-0.1.12-py3-none-any.whl size=16033 sha256=ea62dc54455890136a7ff700b6b6ebeaefef852a394f1f538e76e55870d667e9
  Stored in directory: /root/.cache/pip/wheels/da/7c/3e/296fe3ed4eb3bd713e91dee0d0549f12f316d49939a64bdc96
Successfully built imantics


In [ ]:
# 23

from scipy.spatial import distance

In [ ]:
# 24

from imantics import Polygons, Mask


def show_mask(img, maskList, img_name='img.png'):
    alpha = 0.6
    beta = 1 - alpha
    alphaRaw=alpha*(img)
    betaBuffer=0
    for mask in maskList:
      betaBuffer += beta*(cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR))
      print(mask.shape)
    dst = np.uint8(alphaRaw+betaBuffer)
    plt.figure(figsize=(16, 9))
    plt.title(img_name)
    plt.imshow(dst)
    plt.show()


def get_mask(base_img, return_bbox=False, except_points=[]):
    img = get_transform(train=False)(base_img, {})[0]
    prediction = model(torch.FloatTensor([img.numpy()]).to(device))

    i = 0
    bboxList=[]
    maskList=[]
    while i < len(prediction[0]['boxes']):
        
        is_ok = True

        for point in except_points:
            if distance.euclidean((bboxList[i][0].item(), bboxList[i][1].item()), point) < 40:
                is_ok = False
                break

        if is_ok and prediction[0]['scores'][i]>0.7:
          bboxList.append(prediction[0]['boxes'][i].detach().cpu().numpy())
          maskList.append(prediction[0]['masks'][i, 0].mul(255).byte().cpu().numpy())

        i += 1
    if len(bboxList)==0:
      del bboxList
      return None, None
    
    del prediction

    if return_bbox:
      
      return maskList, bboxList
    
    return maskList, None


def get_mask_segmentation(maskList):
    segmentationList=[]
    for mask in maskList:
      polygons = Mask(mask).polygons()
      segmentationList.append(polygons.segmentation)

    return segmentationList


def show_segmentation(image_path, except_points=[]):
    base_img = cv2.imread(image_path)
    maskList, _ = get_mask(base_img, except_points=except_points)
    show_mask(base_img, maskList)


def get_image_coco_properties(image_path, image_id, except_points=[]):
    base_img = cv2.imread(image_path)
    maskList, bboxList = get_mask(base_img, return_bbox=True, except_points=except_points)

    if maskList is None:
        return None, None

    segmentationList = get_mask_segmentation(maskList)

    annotationList = []
    for index,bbox in enumerate(bboxList):
      annotation = {
        'bbox': [round(bbox[0]), round(bbox[1]), round(bbox[2] - bbox[0]), round(bbox[3] - bbox[1])],
        'id': -1,
        'image_id': image_id,
        'category_id': 91,
        'iscrowd': 0,
        'segmentation': segmentationList[index]
      }
      annotationList.append(annotation)

    image = {
        "date_captured": "",
        "file_name": os.path.basename(image_path),
        "height": base_img.shape[0],
        "id": image_id,
        "license": 0,
        "url": "",
        "width": base_img.shape[1]
    }

    return image, annotationList

In [ ]:
# 25

from tqdm.notebook import tqdm

In [ ]:
# 26

def segment_and_coco_annotate(images_path, except_points=[]):
    images = []
    coco_annotations = []

    for i, image_path in tqdm(enumerate(images_path), total=len(images_path)):
        image, annotationList = get_image_coco_properties(
            images_path[i],
            2000 + i,
            except_points
        )

        if image is None:
            continue
        for annotation in annotationList:
          coco_annotations.append(annotation)
        images.append(image)

    return images, coco_annotations

In [ ]:
# 27

def view_examples(images, coco_annotations, images_dir='drive/MyDrive/VisionProject/Dataset/Data/', n=3):
    indices = np.random.choice([i for i in range(len(images))], n)
    images = np.array(images)
    coco_annotations = np.array(coco_annotations)
    for image in images[indices]:
        print(image)
        img = cv2.imread(os.path.join(images_dir, image['file_name']))
        maskList = []
        for i in range(len(coco_annotation)):
          if coco_annotation[i]["image_id"]==image["id"]:
            print(coco_annotation[i])
            mask = mask_from_coco(img, coco_annotation[i])
            maskList.append(mask*255)
        show_mask(img, maskList, img_name=image['file_name'])

In [ ]:
# 28
import os
import cv2
from tqdm.notebook import tqdm


def get_frames(video_path, save_path):
    video_name = os.path.basename(video_path)
    vidcap = cv2.VideoCapture(video_path)

    success,image = vidcap.read()
    count = 0
    success = True

    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = list(range(0, total_frames, 3))  # какие фреймы нужно вырезать

    for frame_number in tqdm(frames, leave=False):
        vidcap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        ret, frame = vidcap.read()
        cv2.imwrite(os.path.join(save_path, '%s_frame%d.jpg' % (video_name, frame_number)), frame)

In [ ]:
# 29

!mkdir video

In [ ]:
# 30
 
get_frames('/content/drive/MyDrive/VisionProject/Videos/video_1.mp4', './video/')


  0%|          | 0/41 [00:00<?, ?it/s]

In [ ]:
# 31
ls

coco_eval.py   data/   engine.py    __pycache__/  transforms.py  video_1/
coco_utils.py  drive/  labels.json  sample_data/  utils.py       vision/


In [ ]:
# 32
images, coco_annotation = segment_and_coco_annotate(
    glob.glob('./video/*'),
)

  0%|          | 0/41 [00:00<?, ?it/s]

In [ ]:
# 33
print(glob.glob('./video/*'))

['./video/video_1.mp4_frame30.jpg', './video/video_1.mp4_frame15.jpg', './video/video_1.mp4_frame36.jpg', './video/video_1.mp4_frame105.jpg', './video/video_1.mp4_frame60.jpg', './video/video_1.mp4_frame21.jpg', './video/video_1.mp4_frame75.jpg', './video/video_1.mp4_frame24.jpg', './video/video_1.mp4_frame3.jpg', './video/video_1.mp4_frame48.jpg', './video/video_1.mp4_frame102.jpg', './video/video_1.mp4_frame66.jpg', './video/video_1.mp4_frame0.jpg', './video/video_1.mp4_frame63.jpg', './video/video_1.mp4_frame12.jpg', './video/video_1.mp4_frame42.jpg', './video/video_1.mp4_frame27.jpg', './video/video_1.mp4_frame96.jpg', './video/video_1.mp4_frame111.jpg', './video/video_1.mp4_frame39.jpg', './video/video_1.mp4_frame120.jpg', './video/video_1.mp4_frame117.jpg', './video/video_1.mp4_frame90.jpg', './video/video_1.mp4_frame114.jpg', './video/video_1.mp4_frame6.jpg', './video/video_1.mp4_frame93.jpg', './video/video_1.mp4_frame18.jpg', './video/video_1.mp4_frame81.jpg', './video/video_1

In [ ]:
# 34

print(len(images))

In [ ]:
# 35

view_examples(images, coco_annotation, images_dir='./video', n=200)

In [ ]:
# 36

import plotly.express as px

px.imshow(cv2.imread('./video/video_1.mp4_frame108.jpg'))

In [ ]:
# 37

with open('drive/MyDrive/VisionProject/Dataset/Labels.json', 'r') as fw:
    labels = json.load(fw)
    labels['images'].extend(images)
    labels['annotations'].extend(coco_annotation)
    with open('drive/MyDrive/VisionProject/NewDataset/NewLabels.json', 'w') as fw:
        json.dump(labels, fw)

In [ ]:
# 38

ls

coco_eval.py   drive/     __pycache__/  transforms.py  video/
coco_utils.py  engine.py  sample_data/  utils.py       vision/


In [ ]:
# 39
!cp /content/video/* /content/drive/MyDrive/VisionProject/NewDataset/Data/

In [ ]:
# 40

!zip -r segmented_dataser.zip ./video/*
!mv segmented_dataser.zip ./drive/MyDrive/VisionProject/NewDataset/